<a href="https://colab.research.google.com/github/bwr02/ECE180-Final/blob/Barry/ECE180_Final_Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECE 180 Final Project
This is the official notebook that we will present. Please keep it nice and clean.

## Task
- 30 kinds of balls

## Model Architecture
- TBD

## Group members
- Barry Wang
- Ben Richardson
- Allan Garcia
-
-
-

In [2]:
# Generic Data Handling
import numpy as np

# Data Visualization
import matplotlib as plt

# PyTorch Basic Needs
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Add other stuff as we go

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cuda device


#Data Importing and Preprocessing
Import the dataset and group into training, validation, and testing groups. We may need to do cross validation.

In [3]:
# Data importing
!pip install kaggle # Install Kaggle API

In [4]:
# Upload Kaggle API Key
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"allgar","key":"24a4520df8ec60b3365874ac903b1e1f"}'}

In [5]:
# Download the Dataset

# Create a directory for Kaggle and move the API key there
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Use Kaggle API to download the dataset
!kaggle datasets download -d gpiosenka/balls-image-classification

Dataset URL: https://www.kaggle.com/datasets/gpiosenka/balls-image-classification
License(s): CC0-1.0
 97% 140M/144M [00:01<00:00, 141MB/s]
100% 144M/144M [00:01<00:00, 141MB/s]


In [6]:
# Extract the Dataset

# Unzip the dataset
!unzip balls-image-classification.zip -d balls_image_classification

Archive:  balls-image-classification.zip
  inflating: balls_image_classification/EfficientNetB0 model-30-(224 X 224)- 99.33.h5  
  inflating: balls_image_classification/balls.csv  
  inflating: balls_image_classification/test/baseball/1.jpg  
  inflating: balls_image_classification/test/baseball/2.jpg  
  inflating: balls_image_classification/test/baseball/3.jpg  
  inflating: balls_image_classification/test/baseball/4.jpg  
  inflating: balls_image_classification/test/baseball/5.jpg  
  inflating: balls_image_classification/test/basketball/1.jpg  
  inflating: balls_image_classification/test/basketball/2.jpg  
  inflating: balls_image_classification/test/basketball/3.jpg  
  inflating: balls_image_classification/test/basketball/4.jpg  
  inflating: balls_image_classification/test/basketball/5.jpg  
  inflating: balls_image_classification/test/beachballs/1.jpg  
  inflating: balls_image_classification/test/beachballs/2.jpg  
  inflating: balls_image_classification/test/beachballs/3.jpg

In [21]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define data transforms
transform = transforms.Compose([
    transforms.ToTensor()
])

# Load the dataset
train_dataset = datasets.ImageFolder(root='balls_image_classification/train', transform=transform)
val_dataset = datasets.ImageFolder(root='balls_image_classification/valid', transform=transform)
test_dataset = datasets.ImageFolder(root='balls_image_classification/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Defining the Model
Define and compile a CNN model

In [12]:
# Input size is 3 * 224 * 224
class BallClassifierCNN(nn.Module):
    def __init__(self, dropout_rate = 0.2):
        super(BallClassifierCNN, self).__init__()
        # Drop out every other layer
        # Pooling in forward
        # 3 * 224 * 224
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        # 64 * 112 * 112
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        # 128 * 56 * 56
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        # 256 * 28 * 28
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        # 512 * 14 * 14
        # log this FC input size
        self.fc_input_size = 512 * 14 * 14
        self.fc1 = nn.Linear(self.fc_input_size, 1024)
        # 1 * 1024
        self.fc2 = nn.Linear(1024, 512)
        # 1 * 512
        self.fc3 = nn.Linear(512, 30)
        # 1 * 30 output

        # Each pooling layer reduces size by factor of 4
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # 224 -> 112
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # 112 -> 56
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # 56 -> 28
        x = self.pool(F.relu(self.bn4(self.conv4(x))))  # 28 -> 14

        x = x.view(-1, self.fc_input_size)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        return F.log_softmax(x, dim=1)

In [20]:
model = BallClassifierCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
size = len(train_loader.dataset)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

print("Training complete.")

Epoch [1/20], Loss: 5.161000378363956
Epoch [2/20], Loss: 2.7353958007508674
Epoch [3/20], Loss: 2.585742907186525
Epoch [4/20], Loss: 2.3840916146219304
Epoch [5/20], Loss: 2.265254453220199
Epoch [6/20], Loss: 2.0810052093151397
Epoch [7/20], Loss: 1.9486242701522016
Epoch [8/20], Loss: 1.8446041345596313
Epoch [9/20], Loss: 1.714316664543827
Epoch [10/20], Loss: 1.5730206850355706
Epoch [11/20], Loss: 1.5285151205231657
Epoch [12/20], Loss: 1.364204080231422
Epoch [13/20], Loss: 1.3472743609310252
Epoch [14/20], Loss: 1.2955012780375186
Epoch [15/20], Loss: 1.1966561263641424
Epoch [16/20], Loss: 1.1520178761102458
Epoch [17/20], Loss: 1.064179350317052
Epoch [18/20], Loss: 1.030324939605409
Epoch [19/20], Loss: 0.9951905054328716
Epoch [20/20], Loss: 0.9036034780793485
Training complete.


In [22]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

loss_fn = nn.CrossEntropyLoss()
test(test_loader, model, loss_fn)

Test Error: 
 Accuracy: 60.0%, Avg loss: 1.557110 



#Training
Train the model and find the best hyperparams. Let's try to automate this.

In [ ]:
# Have a trainer class that will work on one or a set of models and return the
# best model. Make it so that it can save its progress to a file every defined
# interval.

class Trainer(object):
  def __init__(self, ...):
    pass

  def train(self, ...):
    pass

  def save(self, ...):
    pass

  def recover(self, ...):
    pass


In [ ]:
trainer = Trainer(...)
result = trainer.train(...)

# Testing
Test the model and show test results

In [ ]:
def test(model, img_test, label_test):
  pass